In [71]:
import numpy as np

data = np.load('../processed_data/split_dataset/data.npz', allow_pickle=True)
label = np.load('../processed_data/split_dataset/label.npz.npy')

In [72]:
X = np.stack([data[f'arr_{i}'].item()['three_sec'] for i in range(len(data))])
X_30sec = np.stack([data[f'arr_{i}'].item()['thirty_sec'] for i in range(len(data))])

In [73]:
import numpy as np

def generate_k_fold_dataset(data_3sec, data_30sec, label, k):
    numbers = np.arange(data_3sec.shape[0])
    np.random.shuffle(numbers)
    test_indices = np.array_split(numbers, k)
    
    train_folds, test_3sec_folds, test_30sec_folds = [], [], []
    for i in range(k):
        mask = np.zeros(data_3sec.shape[0], dtype=bool)
        mask[test_indices[i]] = True
        
        X_train, y_train = data_3sec[~mask], label[~mask]
        X_3sec_test, y_3sec_test = data_3sec[mask], label[mask]
        X_30sec_test, y_30sec_test = data_30sec[mask], label[mask]
        
        train_folds.append((X_train, y_train))
        test_3sec_folds.append((X_3sec_test, y_3sec_test))
        test_30sec_folds.append((X_30sec_test, y_30sec_test))
    
    return train_folds, test_3sec_folds, test_30sec_folds
        
train_folds, test_3sec_folds, test_30sec_folds = generate_k_fold_dataset(X, X_30sec, label, 10)

In [74]:
from scipy import stats

def predict_and_mode(model, X_test, scaler):
    X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    results = model.predict(X_test_scaled.reshape(-1, X_test.shape[-1]))
    results = results.reshape(X_test.shape[:-1])
    final_results = stats.mode(results, axis=1)

    return final_results.mode.flatten()

In [75]:
def predict_naive(model, X_test, scaler):
    X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    results = model.predict(X_test_scaled.reshape(-1, X_test.shape[-1]))

    return results

In [76]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def aggregate_metrics(y_true, y_pred):
    return accuracy_score(y_true, y_pred),\
        precision_score(y_true, y_pred, average='macro', zero_division=0.),\
        recall_score(y_true, y_pred, average='macro', zero_division=0.),\
        f1_score(y_true, y_pred, average='macro', zero_division=0.)

In [77]:
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score


def perform_experiment(train_folds, test_3sec_folds, test_30sec_folds, fold_idx,
                       type_return='report'):
    
    if type_return not in ['report', 'metrics']:
        raise ValueError("Invalid return_type. Must be either 'report' or 'metrics'.")
    
    X_train, y_train = train_folds[fold_idx]
    X_test, y_test = test_3sec_folds[fold_idx]
    X_test_30sec, y_test_30sec = test_30sec_folds[fold_idx]

    rand_perm = np.random.permutation(np.arange(X_test.shape[0]))
    X_test, y_test = X_test[rand_perm], y_test[rand_perm]
    X_test_30sec, y_test_30sec = X_test_30sec[rand_perm], y_test_30sec[rand_perm]
    
    y_train = np.repeat(y_train, X_train.shape[1])
    X_train = X_train.reshape(-1, X_train.shape[-1])

    rand_perm = np.random.permutation(np.arange(X_train.shape[0]))
    X_train, y_train = X_train[rand_perm], y_train[rand_perm]
    
    scalar = StandardScaler()
    X_train = scalar.fit_transform(X_train)

    # Change the model here to experiment with different models
    model = SVC(kernel='rbf', decision_function_shape='ovo', C=1)
    model.fit(X_train, y_train)

    if type_return == 'report':
        train_report = classification_report(y_train, model.predict(X_train))
        mode_report = classification_report(y_test, predict_and_mode(model, X_test, scalar))
        naive_report = classification_report(y_test_30sec, predict_naive(model, X_test_30sec, scalar)) 
        return train_report, mode_report, naive_report
    else:
        train_metrics = aggregate_metrics(y_train, model.predict(X_train))
        mode_metrics = aggregate_metrics(y_test, predict_and_mode(model, X_test, scalar))
        naive_metrics = aggregate_metrics(y_test_30sec, predict_naive(model, X_test_30sec, scalar))
        return train_metrics, mode_metrics, naive_metrics

In [78]:
def k_fold_experiment(train_folds, test_3sec_folds, test_30sec_folds):
    metrics = []
    for k in range(len(train_folds)):
        train_metrics, mode_metrics, naive_metrics = perform_experiment(train_folds, test_3sec_folds, test_30sec_folds, k, type_return='metrics')
        print(f"Fold {k+1}:")
        print(f"Train metrics: acc={train_metrics[0]:.3f}, precision={train_metrics[1]:.3f}, recall={train_metrics[2]:.3f}, f1={train_metrics[3]:.3f}")
        print(f"Mode Strategy metrics: acc={mode_metrics[0]:.3f}, precision={mode_metrics[1]:.3f}, recall={mode_metrics[2]:.3f}, f1={mode_metrics[3]:.3f}")
        print(f"Naive Strategy metrics: acc={naive_metrics[0]:.3f}, precision={naive_metrics[1]:.3f}, recall={naive_metrics[2]:.3f}, f1={naive_metrics[3]:.3f}")
        metrics.append((train_metrics, mode_metrics, naive_metrics))
    print(f"Average train metrics: acc={np.mean([m[0][0] for m in metrics], axis=0):.3f}, precision={np.mean([m[0][1] for m in metrics], axis=0):.3f}, recall={np.mean([m[0][2] for m in metrics], axis=0):.3f}, f1={np.mean([m[0][3] for m in metrics], axis=0):.3f}")
    print(f"Average mode metrics: acc={np.mean([m[1][0] for m in metrics], axis=0):.3f}, precision={np.mean([m[1][1] for m in metrics], axis=0):.3f}, recall={np.mean([m[1][2] for m in metrics], axis=0):.3f}, f1={np.mean([m[1][3] for m in metrics], axis=0):.3f}")
    print(f"Average naive metrics: acc={np.mean([m[2][0] for m in metrics], axis=0):.3f}, precision={np.mean([m[2][1] for m in metrics], axis=0):.3f}, recall={np.mean([m[2][2] for m in metrics], axis=0):.3f}, f1={np.mean([m[2][3] for m in metrics], axis=0):.3f}")

In [79]:
k_fold_experiment(train_folds, test_3sec_folds, test_30sec_folds)

Fold 1:
Train metrics: acc=0.949, precision=0.950, recall=0.949, f1=0.949
Mode Strategy metrics: acc=0.790, precision=0.787, recall=0.775, f1=0.774
Naive Strategy metrics: acc=0.770, precision=0.750, recall=0.748, f1=0.740
Fold 2:
Train metrics: acc=0.948, precision=0.949, recall=0.949, f1=0.949
Mode Strategy metrics: acc=0.850, precision=0.848, recall=0.837, f1=0.827
Naive Strategy metrics: acc=0.800, precision=0.783, recall=0.766, f1=0.770
Fold 3:
Train metrics: acc=0.949, precision=0.949, recall=0.948, f1=0.948
Mode Strategy metrics: acc=0.710, precision=0.692, recall=0.753, f1=0.695
Naive Strategy metrics: acc=0.740, precision=0.759, recall=0.757, f1=0.735
Fold 4:
Train metrics: acc=0.950, precision=0.950, recall=0.950, f1=0.950
Mode Strategy metrics: acc=0.780, precision=0.775, recall=0.769, f1=0.758
Naive Strategy metrics: acc=0.770, precision=0.761, recall=0.760, f1=0.746
Fold 5:
Train metrics: acc=0.951, precision=0.951, recall=0.951, f1=0.951
Mode Strategy metrics: acc=0.800, 